![cartoon](images/nmt_cartoon.png)

## seq2seq 应用探索: 从文本摘要到机器翻译

sunoonlee  
2017.5.17


## CONTENTS
**GPU 与数据量**  
**文本摘要探索**  
**翻译模型改进**  
**API 和微信机器人**  

![data](images/data.jpg)

## GPU 与数据量

### 数据量

seq2seq 单次前向传播需要的计算量

![computation](images/computation.png)

-- [Introduction to Neural Machine Translation with GPUs (Part 2)](https://devblogs.nvidia.com/parallelforall/introduction-neural-machine-translation-gpus-part-2/)

### 数据量


> Always train this type of model with GPUs! 

> To get a well-trained model, it takes about three to twelve days using a GeForce GTX Titan X.

-- Kyunghyun Cho, [Introduction to Neural Machine Translation with GPUs (Part 3)](https://devblogs.nvidia.com/parallelforall/introduction-neural-machine-translation-gpus-part-3/)

### 数据量

几周前:

> 1万条语料太大, 笔记本都死了!

现在:

> 10万条语料太少...

### 谷歌云平台

* 单 GPU (Tesla K80) 比自己电脑快 15-20 倍 (矩阵乘法算例)
* 感谢免费账户
![billing](images/gce_billing.png)

![summarize](images/summarize.jpg)

## 文本摘要探索


### 文本摘要 - 语料

* 英文:
  * Gigaword: 会员制, 需收费
  * DUC: 小数据, 用来评测和比赛
* 中文
  * LCSTS: 比较好的语料, 取自新浪微博
    * 发邮件没人理...
  * 搜狗实验室新闻数据: 质量稍差

### 搜狗新闻语料处理

* [搜狗实验室（Sogou Labs）- 搜狐新闻数据](http://www.sogou.com/labs/resource/cs.php)
  * 共 1411996 条语料, 包括字段: url, title, content
* 语料处理:
  * 转换编码(gb18030 -> utf-8), 提取 XML 字段
  * 根据 url 剔除质量较差的语料 (商品介绍, 广告, 论坛帖子...)
  * 控制最大和最小长度
* 最后得到 11w 条语料供训练
* 处理过程见 [textsum/data.ipynb](https://github.com/sunoonlee/DeepLearning101/blob/master/final/textsum/data.ipynb)

### 文本摘要模型训练

* 训练及测验结果: [textsum.ipynb](https://github.com/sunoonlee/DeepLearning101/blob/master/final/textsum/textsum.ipynb)
* 效果不佳
* 答疑结论: 数据量太小. 
* 对策:
  * 1 - 缩小领域, 聚焦某一类新闻  -> 找不到数据
  * 2 - 简化问题, 如 改为提取关键词
  * 3 - 采用 end2end 之外的方法, 如基于句子的 embedding
  * 4 - 弃坑  <-- 我的选择

![translate](images/translate.jpg)
## 翻译模型

### 任务流程

1. Define Task
  - 摘要, 翻译, ...
2. Define Dataset
3. Define metric
  - 摘要 - `ROUGE`, 翻译 - `BLEU`
4. Split dataset
5. Establish a baseline
6. Implement model

-- Project advice from [cs224n-2017-lecture5.pdf](http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture5.pdf)

### 翻译模型 - 语料

* 语料: [联合国平行语料](https://conferences.unite.un.org/UNCorpus). 质量较高.
* 但计算资源仍然有限, 怎么办?
* 语料筛选
  * 只保留长度 6-15 的句子
  * 剔除无意义的噪音句子
* 词表大小和 UNK 比例
  * 英文做 stemming
  * 截断词频取 7~8
  * 再剔除一轮 UNK 比例 > 0.05 的句子
* 处理过程见 [data.ipynb](https://github.com/sunoonlee/DeepLearning101/blob/master/final/data.ipynb)

### 翻译模型 - 学习率

学习率需要小心选择 (都是泪..)

![learningrate](images/learningrates.jpeg)

from: [CS231n](http://cs231n.github.io/neural-networks-3/)

### 翻译模型 - tips

[neubig/nmt-tips: A tutorial about neural machine translation including tips on building practical systems](https://github.com/neubig/nmt-tips)
* 非常好的 tips, 相见恨晚
* 作者用 1w 条语料训练 英文-日文 翻译模型

### 翻译模型 - 指标

* BLEU (bilingual evaluation understudy)

```python
from nltk.translate.bleu_score import corpus_bleu
bleu = corpus_bleu(list_of_references, hypotheses)
```


### 翻译模型 - 结果

* 训练及测试结果见 [seq2seq_nmt.ipynb](https://github.com/sunoonlee/DeepLearning101/blob/master/final/seq2seq_nmt.ipynb)
* 结果勉强可看
* BLEU = 0.282

![walle](images/walle.jpg)

### API 和微信机器人

* 用 `flask-restful` 构建 API
* 用 `wxpy` 方便地接入微信机器人

```python
bot = Bot()
sess = tf.Session()
model = prepare_model(sess, vocab_path, model_path)

@bot.register()
def reply(msg):
    print(msg)
    return model.decode(sess, [msg.text])

bot.join()
```

仅适合于"联合国"风格的短句...

![wechat](images/wechat.jpeg)

### 致谢

感谢 DL101, 让我重新找回了久违的心流体验.  
感谢童老师的精彩讲解和悉心指点.  
感谢课程组和教练们的辛勤付出.  
感谢三个月来共同学习的伙伴们.  